In [7]:
from src.data_loader import load_data
from src.utils import load_config

from src import wikipedia_article_dataset

from sklearn.model_selection import train_test_split
import tiktoken

from torch.utils.data.dataloader import DataLoader

In [9]:
MAX_LENGTH = 400
BATCH_SIZE = 16

In [2]:
config = load_config("just-load")

In [3]:
df = load_data(config["data_loader"], "binary")

In [4]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["label"], test_size=0.2)

In [5]:
tokenizer = tiktoken.get_encoding("cl100k_base")

In [10]:
train_dataset = wikipedia_article_dataset.WikipediaArticleDataset(
    X_train,
    y_train,
    tokenizer.encode,
    MAX_LENGTH
)

test_dataset = wikipedia_article_dataset.WikipediaArticleDataset(
    X_test,
    y_test,
    tokenizer.encode,
    MAX_LENGTH
)

train_dataloader = DataLoader(train_dataset, BATCH_SIZE)
test_dataloader = DataLoader(test_dataset, BATCH_SIZE)